In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'svg'
import networkx
from typing import List, Tuple
import matplotlib.pyplot as plt
import numpy

# Setup

## Specify Graph

In [ ]:
edges: List[Tuple[int, int, float]] = [
    (0, 1, 1.0),
    (0, 2, 1.0),
    (1, 2, 1.0),
    (1, 3, 1.0),
    (2, 3, 1.0),
]

In [ ]:
graph = networkx.Graph()
graph.add_weighted_edges_from(edges)
N = graph.number_of_nodes()

In [ ]:
fig, ax = plt.subplots(1, 1)
positions = networkx.spring_layout(graph, fixed=[0, 3], pos={0: (-0.5, 0), 3: (0.5, 0)})
networkx.draw_networkx_nodes(graph, positions)
networkx.draw_networkx_edges(graph, positions)
networkx.draw_networkx_labels(graph, positions, font_color="white")
networkx.draw_networkx_edge_labels(
    graph, positions, edge_labels=networkx.get_edge_attributes(graph, "weight")
)

fig.set_size_inches(3.2, 2.4)
fig.set_tight_layout(True)
cut = 1.2
xmax = cut * max(x for x, _ in positions.values())
ymax = cut * max(y for _, y in positions.values())
xmin = cut * min(x for x, _ in positions.values())
ymin = cut * min(y for _, y in positions.values())
ax.set_xlim(xmin, xmax)
ax.set_ylim(ymin, ymax)
fig.savefig("graph.svg")

## Setup Problem for Qiskit

First we obtain the weight matrix of the graph:

In [ ]:
weight_matrix = networkx.convert_matrix.to_numpy_array(graph)
weight_matrix

Qiskit provides a handy routine to obtain the Ising Hamiltonian associated with the Maximum-Cut problem. It returns a weighted Ising operator and an energy offset from the constant term.

In [ ]:
from qiskit.optimization.applications.ising import max_cut

hamiltonian, offset = max_cut.get_operator(weight_matrix)
print("Hamiltonian:")
print("------------")
print(hamiltonian)
print("energy offset:", offset)
print(hamiltonian.print_details())

# Solve the Problem

In [ ]:
def plot_solution(solution: List[int]):
    fig, ax = plt.subplots(1, 1)
    colors = ["C1" if solution[i] else "C0" for i in range(N)]
    networkx.draw_networkx_nodes(graph, positions, node_color=colors)
    networkx.draw_networkx_edges(graph, positions)
    networkx.draw_networkx_labels(graph, positions, font_color="white")
    networkx.draw_networkx_edge_labels(
        graph, positions, edge_labels=networkx.get_edge_attributes(graph, "weight")
    )
    fig.set_size_inches(3.2, 2.4)
    fig.set_tight_layout(True)

## Brute-Force

In [ ]:
best_profit = 0.0


def generate_binary_list(index: int) -> List[int]:
    # generate solution candidates (lists of 0's and 1's):
    # 1. bin() converts to binary string
    # 2. [:2] removes the '0b' prefix
    # 3. .zfill(N) prepends 0s until a length of N has been achieved
    return [int(digit) for digit in bin(combination)[2:].zfill(N)]


for combination in range(2 ** N):
    # generate solution candidates (lists of 0's and 1's):
    # 1. bin() converts to binary string
    # 2. [:2] removes the '0b' prefix
    # 3. .zfill(N) prepends 0s until a length of N has been achieved
    binary = generate_binary_list(combination)

    # evaluate the cost function
    profit = 0.0
    for i in range(N):
        for j in range(N):
            profit += weight_matrix[i, j] * binary[i] * (1 - binary[j])

    # check if we found a better solution
    if profit > best_profit:
        best_profit = profit
        solution = binary

    # print info about current combination
    print(
        "combination {}: binary = {}, profit = {}".format(
            combination, str(binary), profit
        )
    )

print()
print("optimal solution: binary = {}, profit = {}".format(str(solution), best_profit))

plot_solution(solution)

## Diagonalize Ising Hamiltonian

In [ ]:
from qiskit.aqua.algorithms import NumPyMinimumEigensolver
from qiskit.optimization.applications.ising.common import sample_most_likely

result_diag = NumPyMinimumEigensolver(hamiltonian).run()
energy_diag = result_diag.eigenvalue.real + offset

state = sample_most_likely(result_diag.eigenstate)
print("ground state energy:", energy_diag)
print("maximum profit:", -energy_diag)
print("most likely binary string:", state)
plot_solution(state)

## VQE on Simulated Quantum Computer

Initialize classical optimization algorithm:

In [ ]:
from qiskit.aqua.components.optimizers import SPSA, COBYLA

optimizer = COBYLA()

Initialize quantum circuit:

In [ ]:
from qiskit.circuit.library import TwoLocal

circuit = TwoLocal(hamiltonian.num_qubits, "ry", "cz", reps=5, entanglement="linear")
circuit.draw(filename="circuit.svg")
circuit

In [ ]:
from qiskit import aqua

aqua.aqua_globals.random_seed = numpy.random.default_rng(498615)
seed = 198687

### Without Noise

Initialize instance of the quantum simulator and set fixed speeds in order to reproduce the results:

In [ ]:
from qiskit.aqua import QuantumInstance
from qiskit.providers.aer import StatevectorSimulator

backend = StatevectorSimulator(method="statevector")

quantum_instance = QuantumInstance(backend, seed_simulator=seed, seed_transpiler=seed)

We create an instance of the VQE algorithm using the Ising Hamiltonian, the classical optimizer and the quantum simulator. Additionally we provide a callback function that gathers metrics for each iteration.

In [ ]:
from qiskit.aqua.algorithms import VQE

means_vqe = []


def callback(evaluations: int, parameters: numpy.ndarray, mean: float, stddev: float):
    means_vqe.append(mean)


vqe = VQE(
    hamiltonian,
    circuit,
    optimizer,
    quantum_instance=quantum_instance,
    callback=callback,
)
result_vqe = vqe.run()

means_vqe = numpy.array(means_vqe)

In [ ]:
fig, ax = plt.subplots(1, 1)
ax.plot(means_vqe + offset, label="VQE")
ax.axhline(energy_diag, color="C1", label="exact")
ax.set_xlabel("iteration")
ax.set_ylabel(r"$\langle H_{\mathrm{MC}}\rangle$")
ax.set_title("VQE Convergence")
ax.legend()
ax.grid(True)
fig.savefig("convergence_vqe.svg")

### With Noise

Qiskit provides noise models that you can use in your simulations. You can either create a custom noise model by specifying readout errors and errors for specific gates or load a realistic model based on one of the IBMQ hardware systems (see the [systems list](https://quantum-computing.ibm.com/systems)). Here we will load the noise model for the IBMQ Santiago quantum computer. At the time of writing, this system has a readout error of $3.33\cdot 10^{-2}$ and a CNOT error of $8.489\cdot 10^{-3}$. The depolarization time is $T_1=111.51\,\mathrm{µs}$ and the dephasing time is $T_2=74.55\,\mathrm{µs}$.

In [ ]:
from qiskit import IBMQ
from qiskit.providers.aer.noise import NoiseModel

provider = IBMQ.load_account()
backend = provider.get_backend("ibmq_santiago")
noise_model = NoiseModel.from_backend(backend)
coupling_map = backend.configuration().coupling_map
basis_gates = noise_model.basis_gates

In [ ]:
from qiskit.providers.aer import QasmSimulator

backend = QasmSimulator(method="statevector", max_parallel_experiments=0)

quantum_instance = QuantumInstance(
    backend,
    seed_simulator=seed,
    seed_transpiler=seed,
    coupling_map=coupling_map,
    noise_model=noise_model,
)

In [ ]:
means_vqe_noise = []
stddevs_vqe_noise = []


def callback(evaluations: int, parameters: numpy.ndarray, mean: float, stddev: float):
    means_vqe_noise.append(mean)
    stddevs_vqe_noise.append(stddev)


vqe = VQE(
    hamiltonian,
    circuit,
    optimizer,
    quantum_instance=quantum_instance,
    callback=callback,
)
result_vqe_noise = vqe.run()

means_vqe_noise = numpy.array(means_vqe_noise)
stddevs_vqe_noise = numpy.array(stddevs_vqe_noise)

In [ ]:
fig, ax = plt.subplots(1, 1)
iteration = [i for i, _ in enumerate(means_vqe_noise)]
ax.fill_between(
    iteration, means_vqe_noise - stddevs_vqe_noise + offset, means_vqe_noise + stddevs_vqe_noise + offset, alpha=0.5
)
ax.plot(iteration, means_vqe_noise + offset, label="VQE")
ax.axhline(energy_diag, color="C1", label="exact")
ax.set_xlabel("iteration")
ax.set_ylabel(r"$\langle H_{\mathrm{MC}}\rangle$")
ax.set_title("VQE Convergence (Noise)")
ax.legend()
ax.grid(True)
fig.savefig("convergence_vqe_noise.svg")

### With Readout Error Mitigation

In [ ]:
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter

backend = QasmSimulator(method="statevector", max_parallel_experiments=0)

quantum_instance = QuantumInstance(
    backend,
    seed_simulator=seed,
    seed_transpiler=seed,
    coupling_map=coupling_map,
    noise_model=noise_model,
    measurement_error_mitigation_cls=CompleteMeasFitter,
)

In [ ]:
means_vqe_mitigation = []
stddevs_vqe_mitigation = []


def callback(evaluations: int, parameters: numpy.ndarray, mean: float, stddev: float):
    means_vqe_mitigation.append(mean)
    stddevs_vqe_mitigation.append(stddev)


vqe = VQE(
    hamiltonian,
    circuit,
    optimizer,
    quantum_instance=quantum_instance,
    callback=callback,
)
result_vqe_mitigation = vqe.run()

means_vqe_mitigation = numpy.array(means_vqe_mitigation)
stddevs_vqe_mitigation = numpy.array(stddevs_vqe_mitigation)

In [ ]:
fig, ax = plt.subplots(1, 1)
iteration = [i for i, _ in enumerate(means_vqe_mitigation)]
ax.fill_between(
    iteration, means_vqe_mitigation - stddevs_vqe_mitigation + offset, means_vqe_mitigation + stddevs_vqe_mitigation + offset, alpha=0.5
)
ax.plot(iteration, means_vqe_mitigation + offset, label="VQE")
ax.axhline(energy_diag, color="C1", label="exact")
ax.set_xlabel("iteration")
ax.set_ylabel(r"$\langle H_{\mathrm{MC}}\rangle$")
ax.set_title("VQE Convergence (Noise)")
ax.grid(True)
fig.savefig("convergence_vqe_mitigation.svg")

In [ ]:
print(result_vqe)

In [ ]:
print(result_vqe_noise)

In [ ]:
print(result_vqe_mitigation)